In [2]:
import glob
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
from tqdm import tqdm
import pandas as pd
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.preprocessing import image

In [3]:
from keras.applications.inception_v3 import InceptionV3

In [4]:
import pickle
import os
import glob

In [5]:


parent_folder = "D:/SEM 7/Core Project/MyCode/"


In [6]:
with open(parent_folder +"VQG/Image-Captioning/data_preprocessing/" + "img2questions_dict.pickle","rb") as fh:
    img2questions_dict = pickle.load(fh)

In [7]:
img_folder = "VQG/Image-Captioning/VQg_Dataset/VQG_Dataset/"

In [8]:
img_files = glob.glob(parent_folder+img_folder+"*.*")

In [9]:
img_file_names = [os.path.basename(x) for x in img_files] ## removing the path part, keeping only the filenames

In [10]:
img_id_file_map = dict()

for img_fil in img_file_names:
    img_id_file_map[str((img_fil.split("."))[0] )] = img_fil 


In [ ]:
def preprocess_input(x):
    x /= 255.
    x -= 0.5
    x *= 2.
    return x

In [ ]:
def preprocess(image_path):
    img = image.load_img(image_path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    x = preprocess_input(x)
    return x

In [ ]:
model = InceptionV3(weights='imagenet')

In [ ]:
from keras.models import Model

new_input = model.input
hidden_layer = model.layers[-2].output

model_new = Model(new_input, hidden_layer)

In [ ]:
def encode(image):
    image = preprocess(image)
    temp_enc = model_new.predict(image)
    temp_enc = np.reshape(temp_enc, temp_enc.shape[1])
    return temp_enc

In [ ]:
import os

In [ ]:
problematic_images=''

encoding_img = {}

for imgid,img in tqdm(img_id_file_map.items()):
    
    try:
        encoded = encode(parent_folder+img_folder+img)
        
        encoding_img[imgid] = encoded
        
    except:
        print(img)
        problematic_images += img+"\n"


In [ ]:

with open(parent_folder+"VQG/Image-Captioning/VQg_Dataset/"+"img_vectors.pickle","wb") as fh:
    pickle.dump(encoding_img,fh)
    

In [11]:
with open(parent_folder+"VQG/Image-Captioning/VQg_Dataset/"+"img_vectors.pickle","rb") as fh:
    encoding_img=pickle.load(fh)


In [ ]:
test_sample=encoding_img['208252'].reshape(1,2048)

In [12]:
list_encoding_img = dict((k,i) for i,k in enumerate(encoding_img.keys()))

In [13]:
rev_list_encoding_img = dict((i,k) for i,k in enumerate(encoding_img.keys()))

In [14]:
list_of_imgs=[]

In [15]:
for idx in range(len(list(encoding_img.keys()))):
    
    img_ide = rev_list_encoding_img[idx]
    
    vector = encoding_img[img_ide]
    
    list_of_imgs.append(vector)
    

In [16]:
from sklearn.decomposition import PCA
pca = PCA(n_components=300)
compressed_encoding_img_values = pca.fit_transform(np.vstack(list_of_imgs))

In [17]:
compressed_encoding_img=dict()

In [18]:
for idx in range(len(list(encoding_img.keys()))):
    
    img_ide = rev_list_encoding_img[idx]
    
    vector = compressed_encoding_img_values[idx]
    
    compressed_encoding_img[img_ide] = vector

    

In [19]:
import pickle
with open("D:/MyCodes/EMBEDDINGS/glove_840B_300d.pickle","rb") as fh:
    embed_dict=pickle.load(fh)

In [20]:
embedding_dim = 300

In [21]:
final_vectors=[]

for idx in range(len(list(encoding_img.keys()))):
    
    img_ide = rev_list_encoding_img[idx]
    
#     img_vector = encoding_img[img_ide]
    
    quess = img2questions_dict[img_ide]
    
    img_vector = compressed_encoding_img[img_ide]
    
    for ques in quess:
        
        ques = ques[:-1]
        ques = ques.lower()
        
        ques = ques.split()
        
        sent_v=np.zeros((embedding_dim),dtype='double')
        for word in ques:
            try:
                v = embed_dict[word]
            except:
                v=np.zeros((embedding_dim),dtype='double')

            sent_v+=v
            
        sent_v = sent_v / (len(ques))
                
        
        final_vectors.append([img_vector,sent_v])

In [22]:
with open("img_ques_pair_vectors.pickle","wb") as fh:
    pickle.dump(final_vectors,fh)

In [23]:
with open("img_ques_pair_vectors.pickle","rb") as fh:
    final_vectors=pickle.load(fh)

In [24]:
imgs_matrix = []
ques_matrix = []

In [25]:
for img_vec,ques_vec in final_vectors:
    imgs_matrix.append(img_vec)
    ques_matrix.append(ques_vec)

In [26]:
imgs_matrix = np.vstack(imgs_matrix)
ques_matrix = np.vstack(ques_matrix)

In [27]:
with open("imgs_matrix.pickle","wb") as fh:
    pickle.dump(imgs_matrix,fh)

In [28]:
with open("ques_matrix.pickle","wb") as fh:
    pickle.dump(ques_matrix,fh)

In [29]:
with open("ques_matrix.pickle","rb") as fh:
    ques_matrix = pickle.load(fh)